# <center><font color=navy>DSPG 2023</font></center>
## <center>A Crash Course on Remote Sensing with Google Earth Engine (GEE) in Python</center>
### <center>Naveen Abedin</center>




<center> Research Associate, Data Science for Public Good (DSPG) 2023 </center>
<center> Virginia Tech</center>
<center> naveenabedin@vt.edu </center> 

Much of the resources for this workshop have been gathered from the lecture notes of Dr. Elinor Benami, Fall 2022 (elinor@vt.edu)


This crash-course is designed to briefly introduce applied concepts in remote sensing and using Google Earth Engine in the Jupyter Notebook (Python) environment. 


## Remote Sensing


* Remote sensing is the study of acquiring information about objects or phenomena on the Earth's surface without direct physical contact.

* It involves the use of sensors, typically mounted on satellites or aircraft, to gather data from a distance. These sensors capture electromagnetic radiation reflected or emitted by the Earth's surface, atmosphere, and other objects. 

* Remote sensing technology allows scientists and researchers to obtain valuable information about the Earth's land, water, and atmosphere on a global scale. It provides a way to study and monitor various aspects of the environment, such as land cover, vegetation health, urban development, atmospheric conditions, and natural disasters. 

* This information can be presented in the form of **digital imagery**. 



> Goals:

- Learn about the basic concept of remote sensing
- Use Google Earth Engine platform in the Jupyter Notebook (Python) environment
- Generate some simple maps in R

## Sensors and satellites


Three most well-known satellite missions are:
- MODIS (https://modis.gsfc.nasa.gov/about/)
- Landsat (https://www.nasa.gov/mission_pages/landsat/overview/index.html)
- Sentinel (https://sentinel.esa.int/web/sentinel/missions/sentinel-1)




**MODIS** (Moderate Resolution Imaging Spectroradiometer) is a key instrument onboard two NASA satellites, Terra and Aqua. It is designed to provide accurate and comprehensive observations of the Earth's land, ocean, and atmospheric conditions. 


The **Landsat** program is a series of Earth observation satellites jointly operated by NASA (National Aeronautics and Space Administration) and the USGS (United States Geological Survey). The satellites within the Landsat program have been providing continuous global coverage of the Earth's surface since 1972, making it one of the longest-running satellite programs for Earth observation.



The **Sentinel** satellites are a series of Earth observation satellites developed by the European Space Agency (ESA) as part of the European Union's Copernicus program. The Copernicus program aims to provide accurate and timely environmental information for monitoring and managing the Earth's resources and ecosystems. 



Several types of images can be collected through remote sensing. In this workshop, we will focus on optical and radar imagery. 
**Optical imagery** relies on the reflection and absorption of sunlight to capture images. It utilizes the visible and near-infrared portions of the electromagnetic spectrum. On the other hand, **radar imagery** uses active sensing, where radar systems emit their own electromagnetic waves and measure the backscattered signals that bounce back from the Earth's surface.


(See Dr. Benami's 'Lab 1 - Remote Sensing Basics' for more detail)


## Optical Imagery


Optical images are captured by sensors that analyze the intensity and spectral characteristics of visible and near-infrared light. They can provide high-resolution imagery and the color information in optical images allows for visual interpretation of features. 


<img src="elecspec1.png"  width="600">
</div>

(Image acquired from 'Lecture2_Resolution_EMSpectrum_Fall22', Elinor Benami)

Spectral Reflectance varies across different surfaces:

<img src="spectralreflectance.png"  width="600">
</div>

(Image acquired from 'Lecture2_Resolution_EMSpectrum_Fall22', Elinor Benami)


- Normalized Difference Vegetation Index (NDVI) 
- Enhanced Vegetation Index (EVI)

(visit: https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD13A2)


- Land Surface Temperature (LST)

(visit: https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD11A2)

## Google Earth Engine

Here are some useful definitions and functions to get started on GEE:

1. Shapefiles are a popular geospatial vector data format used in Geographic Information System (GIS) software. They consist of multiple files that collectively store geometric and attribute information about geographic features. In GEE, shapefiles get converted to **geometry** (points, lines, polygons).

2. FeatureCollection: These are a set of geometry that share a common theme, e.g. we can have a FeatureCollection of polygons representing divisions of Bangladesh. 

3. Image: A single image that also contains value (information)

4. ImageCollection: A "stack" or sequence of images with the same attributes

(See Dr. Benami's 'PreLab - Intro to GEE' for more detail)

In [1]:
import ee
import geemap

#import geopandas as gpd
import pandas as pd

ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AfJohXkdgmMUi_L2Lr0g7Sl1fe5q8ZbQvhS2Zvuum78M2_plh0XPVOVr6JI

Successfully saved authorization token.


In [2]:
bangladesh = ee.FeatureCollection('projects/ee-naveenabedin/assets/bgd_admbnda_adm1_bbs_20201113') #ee calls functions from GEE; FeatureCollection calls the shapefile from GEE

Map = geemap.Map() #geemap is the package in Python that enables us to open the GEE map in Python


Map.centerObject(bangladesh, 7) #7 means zoom = 7
Map.addLayer(bangladesh, {}, 'BGD Divisions')

Map

Map(center=[23.827491885464603, 90.2877797027878], controls=(WidgetControl(options=['position', 'transparent_b…

In [3]:
image_collection_name = "MODIS/061/MOD13A2"
date_start = '2016-01-01'
date_end = '2016-12-31'
name = 'MODIS EVI'

# several images COLLECTION
image_collection = (
    ee.ImageCollection(image_collection_name)
    .filter(ee.Filter.date(date_start, date_end)).select('EVI')
)

# For visualization purposes
image = image_collection.max() #max EVI in year 2018
image = image.clip(bangladesh)


palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301']

vizParams = {
    'min': 0, 
    'max': 9000,
    'palette': palette
}

Map.addLayer(image, vizParams, 'BGD EVI')
Map


Map(bottom=14450.0, center=[23.827491885464603, 90.2877797027878], controls=(WidgetControl(options=['position'…

In [4]:
image_collection_name = "MODIS/061/MOD13A2"
date_start = '2018-01-01'
date_end = '2018-12-31'
name = 'MODIS EVI'

# several images COLLECTION
image_collection = (
    ee.ImageCollection(image_collection_name)
    .filter(ee.Filter.date(date_start, date_end)).select('EVI')
)

evi_ts = image_collection.toBands()

output = 'EVI_BGDDiv_2018.csv'
geemap.zonal_statistics(evi_ts, bangladesh, output, statistics_type='MEAN', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\naveenabedin\Desktop\DSPG 2023_1\RS Crash Course\EVI_BGDDiv_2018.csv


In [4]:
#Let's try with Hanover County parcels
hanover = ee.FeatureCollection('projects/ee-naveenabedin/assets/Hanover_Parcels')

Map = geemap.Map()


Map.centerObject(hanover, 10)
Map.addLayer(hanover, {}, 'Hanover Parcels')

Map


Map(center=[37.76140727210689, -77.49240501569922], controls=(WidgetControl(options=['position', 'transparent_…

In [5]:
#Visualize mean EVI in 2020
image_collection_name = "MODIS/061/MOD13A2"
date_start = '2020-01-01'
date_end = '2020-12-31'
name = 'MODIS EVI'

# several images COLLECTION
image_collection = (
    ee.ImageCollection(image_collection_name)
    .filter(ee.Filter.date(date_start, date_end)).select('EVI')
)

# For visualization purposes
image = image_collection.max()
image = image.clip(hanover)


palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301']

vizParams = {
    'min': 0, 
    'max': 9000,
    'palette': palette
}

Map.addLayer(image, vizParams, 'Hanover EVI')
Map


Map(bottom=101637.0, center=[37.76140727210689, -77.49240501569922], controls=(WidgetControl(options=['positio…

In [7]:
image_collection_name = "MODIS/061/MOD13A2"
date_start = '2020-01-01'
date_end = '2020-12-31'
name = 'MODIS EVI'

# several images COLLECTION
image_collection = (
    ee.ImageCollection(image_collection_name)
    .filter(ee.Filter.date(date_start, date_end)).select('EVI')
)

evi_ts = image_collection.toBands()

output = 'EVI_HanPar_2020.csv'
geemap.zonal_statistics(evi_ts, hanover, output, statistics_type='MEAN', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\naveenabedin\Desktop\DSPG 2023_1\RS Crash Course\EVI_HanPar_2020.csv


In [6]:
bangladesh = ee.FeatureCollection('projects/ee-naveenabedin/assets/bgd_admbnda_adm1_bbs_20201113') #ee calls functions from GEE; FeatureCollection calls the shapefile from GEE

Map = geemap.Map() #geemap is the package in Python that enables us to open the GEE map in Python


Map.centerObject(bangladesh, 7) #7 means zoom = 7
Map.addLayer(bangladesh, {}, 'BGD Divisions')

Map

Map(center=[23.827491885464603, 90.2877797027878], controls=(WidgetControl(options=['position', 'transparent_b…

In [7]:
#CHIRPS 
image_collection_name = "UCSB-CHG/CHIRPS/PENTAD"
date_start = '2017-01-01'
date_end = '2017-12-31'
name = 'CHIRPS'

image_collection = (
    ee.ImageCollection(image_collection_name)
    .filter(ee.Filter.date(date_start, date_end)).select('precipitation')
)

image = image_collection.sum()
image = image.clip(bangladesh)


palette = ['#ffffcc','#a1dab4','#41b6c4','#2c7fb8','#253494']

vizParams = {
    'min': 0, 
    'max': 4000,
    'palette': palette #, 
    #'opacity': 0.6
}

Map.addLayer(image, vizParams, 'CHIRPS Rainfall')
Map

Map(bottom=14450.0, center=[23.827491885464603, 90.2877797027878], controls=(WidgetControl(options=['position'…

In [13]:
image_collection_name = "UCSB-CHG/CHIRPS/PENTAD"
date_start = '2017-01-01'
date_end = '2017-12-31'
name = 'CHIRPS'

image_collection = (
    ee.ImageCollection(image_collection_name)
    .filter(ee.Filter.date(date_start, date_end)).select('precipitation')
)

precip_ts = image_collection.toBands()

output = 'PRECIP_Bangladesh2017.csv'
geemap.zonal_statistics(precip_ts, bangladesh, output, statistics_type='MEAN', scale=5000) #average of all pixel values in a particular polygon